In [1]:
# suppress tensorflow logging, usually not useful unless you are having problems with tensorflow or accessing gput
# it seems necessary to have this environment variable set before tensorflow is imported, or else it doesn't take effect
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

# Check GPU Status / Availability

On linux systems you can and should use the `nvidia-smi` tool to check that the gpu is visible, is active and has drivers installed.  You can run the command from a terminal like the following cell.

I also find the following commands useful to monitor the gpu performance from the command line

```
# use watch so basic nvidia-smi redraws at top of screen each second
$ watch -n 1 nvidia-smi

# nvtop is basiclly like top for nvidia gpu
$ sudo apt install nvtop
$ nvtop

# nvitop is similar, gives about same information, but some may prefer this one
$ sudo apt install nvitop
$ nvitop
```

In [2]:
!nvidia-smi

Thu Aug  1 15:34:02 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 555.42.02              Driver Version: 555.42.02      CUDA Version: 12.5     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce GTX 1080 Ti     Off |   00000000:00:08.0 Off |                  N/A |
| 25%   26C    P8              9W /  250W |   10551MiB /  11264MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# GPU to TF/Keras Availability

We can check that tensorflow recognizes the presence of a GPU device as follows.

In [3]:
print('Available Devices : ', tf.config.list_physical_devices())
print('Num GPUs Available: ', len(tf.config.list_physical_devices('GPU')))

Available Devices :  [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Num GPUs Available:  1


# Test Training Performance with only CPU

We can compare performance with and without the gpu.  First we will train a model using only the cpu and record time it takes to train model.  We just train a simple dens NN on mnist data to test 
performance on fitting a model.

In [4]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0


loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

with tf.device('/CPU:0'):
    model_cpu_base = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(1024, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
    ])


In [5]:
def fit_cpu():
    with tf.device('/CPU:0'):
        model_cpu = tf.keras.models.clone_model(model_cpu_base)
        model_cpu.compile(optimizer='adam',
                      loss=loss_fn,
                      metrics=['accuracy'])
        model_cpu.fit(x_train, y_train, epochs=5)
        print('=' * 70)
        
%timeit fit_cpu()

Epoch 1/5
1875/1875 [==============================] - 53s 28ms/step - loss: 0.3331 - accuracy: 0.9002
Epoch 2/5
1875/1875 [==============================] - 52s 28ms/step - loss: 0.0921 - accuracy: 0.9720
Epoch 3/5
1875/1875 [==============================] - 50s 27ms/step - loss: 0.0578 - accuracy: 0.9817
Epoch 4/5
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0470 - accuracy: 0.9845
Epoch 5/5
1875/1875 [==============================] - 49s 26ms/step - loss: 0.0354 - accuracy: 0.9881
Epoch 1/5
1875/1875 [==============================] - 49s 26ms/step - loss: 0.3334 - accuracy: 0.8995
Epoch 2/5
1875/1875 [==============================] - 53s 28ms/step - loss: 0.0917 - accuracy: 0.9720
Epoch 3/5
1875/1875 [==============================] - 53s 28ms/step - loss: 0.0603 - accuracy: 0.9814
Epoch 4/5
1875/1875 [==============================] - 52s 28ms/step - loss: 0.0436 - accuracy: 0.9860
Epoch 5/5
1875/1875 [==============================] - 54s 29ms/step - lo

# Test Training Performance with GPU

Now do same model and fit it on the gpu device.

In [6]:
# recreate a random starting model and compile it
with tf.device('/GPU:0'):
    model_gpu_base = tf.keras.models.Sequential([
      tf.keras.layers.Flatten(input_shape=(28, 28)),
      tf.keras.layers.Dense(1024, activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(10)
    ])


In [7]:
def fit_gpu():
    with tf.device('/GPU:0'):
        model_gpu = tf.keras.models.clone_model(model_gpu_base)
        model_gpu.compile(optimizer='adam',
                      loss=loss_fn,
                      metrics=['accuracy'])
        model_gpu.fit(x_train, y_train, epochs=5)
        print('=' * 70)

%timeit fit_gpu()

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3319 - accuracy: 0.9028
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0932 - accuracy: 0.9705
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0644 - accuracy: 0.9798
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0441 - accuracy: 0.9855
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0386 - accuracy: 0.9875
Epoch 1/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.3212 - accuracy: 0.9043
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0898 - accuracy: 0.9713
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0591 - accuracy: 0.9808
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0450 - accuracy: 0.9854
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0360 - accurac